In [1]:
 ## QUESTAO 1...

In [14]:
import numpy as np
import operator
import pandas as pd
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

In [15]:
import csv
 
def loadCsv(filename):
    lines = csv.reader(open(filename, "r"))
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [x for x in dataset[i]]
    return dataset

In [16]:
def separaDados(data):
        
    data = np.array(data)
    [m, n] = data.shape
    Y = np.array([data[i, n-1] for i in range(0, m)])
    data = np.delete(data, (n-1), axis=1)

    return [data, Y]

In [17]:
def separaDadosTreino(data, rate):
    [xtr, xte] = splitDataset(data, rate)
    [xtr, ytr] = separaDados(xtr)
    [xte, yte] = separaDados(xte)
    return xtr, xte, ytr, yte

In [18]:
import random
def splitDataset(dataset, splitRatio):
    trainSize = int(len(dataset) * splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet, copy]

In [19]:
import numpy as np

class bayes:
    
    def __init__(self, data, y, prefix):
        data = np.array(data)
        self.prefix = np.array(prefix)
        
        self.data = data
        self.Y = y
         
        [self.m , self.n] = np.array(self.data).shape
        self.findClasses()
     
    
    def findClasses(self):
        dic = {}
        for y in self.Y:
            dic[y] = 0
        self.classes = dic.keys()
        
    def mymeshmatrix(self, prefix, sufix):
        v = {}
        prefix = np.array(prefix)
        sufix = np.array(sufix)
        
        for i in range(0, len(prefix)):
            for j in range(0, len(sufix)):
                v[str(prefix[i]) + '_' + str(sufix[j, i])] = float(0)
                
        return v
    
    def mymesh(self, prefix, sufix):
        v = {}
        prefix = np.array(prefix)
        sufix = np.array(sufix)
        
        for i in range(0, len(prefix)):
            for j in range(0, len(sufix)):
                v[str(prefix[i]) + '_' + str(sufix[j])] = float(0)
        return v
    
    def separaDados(self, data):
        
        data = np.array(data)
        [m, n] = data.shape
        Y = np.array([data[i, n-1] for i in range(0, m)])
        data = np.delete(data, (n-1), axis=1)
        
        return [data, Y]

    def find(self,pos, xi, classe):
        # retorna a prob de xi com o sufix em pos
        return self.dicCond
    
    def showCond(self):
        print(self.dicCond)

    def probClass(self, classe):
        return self.dictClass[classe]

    def probCond(self,pref, xi, classe):
        return self.dicCond[pref + '_' + xi + '_' + classe]
    
    def prob(self, i, xi):
        return self.dicAtr[self.prefix[i] + '_' + xi]
              
    def fit(self):
        # criar os dicionários...

        #self.mymesh deve gsrsntir que todos os estados de cada atributo devem ser catalogados
        self.dicAtr = self.mymeshmatrix(self.prefix, self.data)
        self.dicCond = self.mymesh(self.dicAtr.keys(), self.Y)
        self.dictClass = {x:float(0) for x in self.Y}
        
        #contando...
        for i in range(0, self.m):
            self.dictClass[ self.Y[i] ] = self.dictClass[ self.Y[i] ] + 1
            for j in range(0, self.n):
                key = (self.prefix[j]) + '_' + (self.data[i,j])
                self.dicAtr[key] = (self.dicAtr[key]) + 1
                key2 = key + '_' + str(self.Y[i])
                
                self.dicCond[key2] = (self.dicCond[key2]) + 1
         
        # achando a probabilidade de x.e e classe.e e de x.e | classe.e
        m = len(self.data)

        dcond = {}
        dclass = {}
        datr = {}
        
        for key1 in self.dicAtr.keys():
            for key2 in self.dictClass.keys():
                dcond[str(key1) + '_' + str(key2)] = self.dicCond[key1 + '_' + key2]/self.dictClass[key2]
        
        for key in self.dictClass.keys():
            dclass[key] = self.dictClass[key]/self.m

        for key in self.dicAtr.keys():
            datr[key] = self.dicAtr[key]/self.m

        self.dicAtr = datr
        self.dictClass = dclass
        self.dicCond = dcond
        
        for key in self.dicCond.keys():
            if self.dicCond[key] == 0:
                self.dicCond[key] = 0.00001
                
        
        for key in self.dictClass.keys():
            if self.dictClass[key] == 0:
                self.dictClass[key] = 0.00001
        
        
    def predictOne(self, x):
        #x eh uma linha com varios estados de atributos...
        # para cada estado de x, achar p(x.e), p(x.e | classe) e p(classe)
        
        maxprob = 0
        maxclasse = 'none'

        for classe in self.classes:
            
            pc = self.probClass(classe) # probabilidade de cada classe...
            p = [ self.prob(i, x[i]) for i in range(0, len(x)) ]
            pxc = [self.probCond(self.prefix[i], x[i], classe) for i in range(0, len(x))] #probabilidade condicional p(x | c)
            
            prob = reduce(operator.mul, pxc, 1) * (pc ** len(x)) # p(c | x) = produtorio(p(x|c)) * pc
            z = reduce(operator.mul, p, 1) # p(c | x) = produtorio(p(x|c)) * pc / produtorio(p(x))
            
            prob = prob
            if maxprob < prob:
                maxprob = prob
                maxclasse = classe
                
        return maxclasse
    
    def predict(self, data):
            
        return [self.predictOne(np.array(data)[i, :]) for i in range(0, np.array(data).shape[0])]
   
    
    

In [20]:
import pandas as pd

data = loadCsv('carData.csv')
pref = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']
data = np.array(data)

# ja que as listas do python sao um cu, vamo usar numpy

pref = np.array(pref)
pref = np.append(pref, 'class')
pf = pd.DataFrame(data, columns= pref)

pf.head()

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [29]:
# Utilizei dicionarios para calcular a probabilidade de cada valor
# de atributo e seu condicional com determinado estado de uma classe.
import numpy as np

# preciso que seja um np.array
data = np.array(loadCsv('carData.csv'))

#separando os dados...
X_train, X_test, y_train, y_test = separaDadosTreino(data, 0.7)

#prefixos de cada atributo
pref = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']

bay = bayes(X_train, y_train, pref)

bay.fit()

[mte, nte] = np.array(X_test).shape

y_pred = bay.predict(X_test)

erros = 0.0
acertos = 0.0
print(" Naive-Bayes - Minha Versao")
print(("tamanho dos dados de treino: {0}").format(len(X_train)))
print(("tamanho dos dados de teste: {0}").format(len(X_test)))

print("Total acuracia: {}%".format(100 * accuracy_score(y_true=y_test, y_pred=y_pred)))
print(classification_report(y_true=y_test, y_pred=y_pred, target_names=["unacc", "acc", "good", "vgood"]))

 Naive-Bayes - Minha Versao
tamanho dos dados de treino: 1209
tamanho dos dados de teste: 519
Total acuracia: 70.5202312139%
             precision    recall  f1-score   support

      unacc       0.00      0.00      0.00       120
        acc       0.00      0.00      0.00        16
       good       0.71      1.00      0.83       366
      vgood       0.00      0.00      0.00        17

avg / total       0.50      0.71      0.58       519



In [23]:
 # QUESTAO 2...

In [24]:

# Carregando o Dataset...
data = pd.read_csv("carData.csv", header=None)

for i in range(0, data.shape[1]):
    data.iloc[:,i] = LabelEncoder().fit_transform(data.iloc[:,i])

# Separação do Conjunto de Treino e Conjunto de Teste 70% treino, 30% teste
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:,:-1], data.iloc[:,-1], test_size=0.3)


In [25]:
clf = MultinomialNB()
clf.fit(X_train.values, y_train.values)

y_pred = clf.predict(X_test.values)

print("Naive-Bayes (Versao SKlearn)")
print("Total Accuracy: {}%".format(100 * accuracy_score(y_true=y_test, y_pred=y_pred)))

print("\nClassification Report:")
print(classification_report(y_true=y_test, y_pred=y_pred, target_names=["unacc", "acc", "good", "vgood"]))



Naive-Bayes (Versao SKlearn)
Total Accuracy: 69.5568400771%

Classification Report:
             precision    recall  f1-score   support

      unacc       0.50      0.02      0.04       109
        acc       0.00      0.00      0.00        23
       good       0.70      1.00      0.82       360
      vgood       0.00      0.00      0.00        27

avg / total       0.59      0.70      0.58       519



In [26]:
# QUESTAO 3

In [28]:

print("Minha solução usa um processo de contagem de cada estado de um atributo em relação a um estado de classe, supondo que os atributos não são co-relacionados. A acurácia foi em torno de 72% enquanto que a versão do scikit-learn foi em torno de 72% também")


Minha solução usa um processo de contagem de cada estado de um atributo em relação a um estado de classe, supondo que os atributos não são co-relacionados. A acurácia foi em torno de 72% enquanto que a versão do scikit-learn foi em torno de 72% também
